In [ ]:
import numpy as np
import pickle
import sys
sys.path.append("../../../assets")
from models import RandomWalkDDM, MixtureRandomWalkDDM, LevyFlightDDM, RegimeSwitchingDDM

# Constants

In [ ]:
NUM_TRAINING_SIMULATIONS = 5000
NUM_VALIDATION_SIMULATIONS = 200

# Data Simulation

In [ ]:
# initialize generative models
random_walk_ddm = RandomWalkDDM()
mixture_random_walk_ddm = MixtureRandomWalkDDM()
levy_flight_ddm = LevyFlightDDM()
regime_switching_ddm = RegimeSwitchingDDM()

## Training Data

In [ ]:
%%time
training_data_m1 = random_walk_ddm.generate(NUM_TRAINING_SIMULATIONS)
training_data_m2 = mixture_random_walk_ddm.generate(NUM_TRAINING_SIMULATIONS)
training_data_m3 = levy_flight_ddm.generate(NUM_TRAINING_SIMULATIONS)
training_data_m4 = regime_switching_ddm.generate(NUM_TRAINING_SIMULATIONS)

In [ ]:
training_data = {
    'model_outputs': [
        {'sim_data': training_data_m1['sim_data'][:, :, None].astype(np.float32)},
        {'sim_data': training_data_m2['sim_data'][:, :, None].astype(np.float32)},
        {'sim_data': training_data_m3['sim_data'][:, :, None].astype(np.float32)},
        {'sim_data': training_data_m4['sim_data'][:, :, None].astype(np.float32)}
        ],
    'model_indices': np.arange(4)
}

In [ ]:
with open('../data/training_data.pkl', 'wb') as f:
    pickle.dump(training_data, f)

## Validation Data

In [ ]:
%%time
validation_data_m1 = random_walk_ddm.generate(NUM_VALIDATION_SIMULATIONS)
validation_data_m2 = mixture_random_walk_ddm.generate(NUM_VALIDATION_SIMULATIONS)
validation_data_m3 = levy_flight_ddm.generate(NUM_VALIDATION_SIMULATIONS)
validation_data_m4 = regime_switching_ddm.generate(NUM_VALIDATION_SIMULATIONS)

In [ ]:
validation_data = {
    'model_outputs': [
        {'sim_data': validation_data_m1['sim_data'][:, :, None].astype(np.float32)},
        {'sim_data': validation_data_m2['sim_data'][:, :, None].astype(np.float32)},
        {'sim_data': validation_data_m3['sim_data'][:, :, None].astype(np.float32)},
        {'sim_data': validation_data_m4['sim_data'][:, :, None].astype(np.float32)}
        ],
    'model_indices': np.arange(4)
}

In [ ]:
with open('../data/validation_data.pkl', 'wb') as f:
    pickle.dump(validation_data, f)